# Download ICESAT-2 Data

Using the icepyx (https://icepyx.readthedocs.io) python module, we can easily download the ATL03 ICESAT-2.

## 1. Import Modules

In [ ]:
import os
import icepyx
import rsgislib.vectorattrs
import rsgislib.vectorgeoms
import rsgislib.vectorutils
import rsgislib.tools.mapping
import rsgislib.tools.geometrytools
import geopandas
%matplotlib inline
import matplotlib.pyplot as plt

## 2. Get list Region of Interests

A set of regions of interest (ROIs) has been defined within the vector layer (`is2_dwnld_rois.geojson`) as polygons using the WGS84 coordinate system (EPSG:4326). It is for these regions that ICESAT-2 data will be downloaded.

### 2.1 Visualise the Input Data

In [ ]:
vec_roi_file = "../data/is2_bathy/is2_dwnld_rois.geojson"
vec_roi_lyr = "is2_dwnld_rois"


In [ ]:
# Define bounding box for map.
bbox = rsgislib.vectorutils.get_vec_layer_extent(vec_roi_file, vec_roi_lyr)
bbox = rsgislib.tools.geometrytools.buffer_bbox(bbox, buf = 5)

# Create a plot using the matplotlib library
fig, ax = plt.subplots(figsize=(5, 5))

# Display countries layer to provide background context
countries_df = geopandas.read_file("../data/country.geojson")
rsgislib.tools.mapping.create_vec_lyr_map(ax, countries_df.boundary, bbox)

# Display the ROI file to check where you will be downloading data for.
roi_polys_df = geopandas.read_file(vec_roi_file)
rsgislib.tools.mapping.create_vec_lyr_map(ax, roi_polys_df, bbox, vec_fill_clrs="red")


### 2.2 See the ROI attribute table

Note, the `name` column will be used for the output directory name when we come to download the data

In [ ]:
roi_polys_df

### 2.3 Get the list of Names and BBOXs

In [ ]:
names = rsgislib.vectorattrs.read_vec_column(vec_file=vec_roi_file, vec_lyr=vec_roi_lyr, att_column="name")
bboxes = rsgislib.vectorgeoms.get_geoms_as_bboxs(vec_file = vec_roi_file, vec_lyr = vec_roi_lyr)

## 3. Download ICESAT-2 Data

The next block of code iterates through the sites defined within the ROIs vector layer, creates a set of directories for the data to be downloaded to and then runs the download - this could take a while depending on your internet connection. It also requires that an Earth Data login (https://www.earthdata.nasa.gov).

In [ ]:
# Define the base directory where data will be downloaded to.
base_data_dir = "icesat2_dwnld"
if not os.path.exists(base_data_dir):
    os.mkdir(base_data_dir)
    
for name, bbox in zip(names, bboxes):
    print(f"{name}: {bbox}")

    # Query BBOX: [lon_LL, lat_LL, lon_UR, lat_UR]
    query_bbox = [bbox[0], bbox[2], bbox[1], bbox[3]]
    
    # Define the directory for this ROI
    roi_data_dir = os.path.join(base_data_dir, name)
    if not os.path.exists(roi_data_dir):
        os.mkdir(roi_data_dir)
        
    # Loop through the years you want to download
    for year in [2019, 2020]:
        print(f"\t{year}")
        out_data_dir = os.path.join(roi_data_dir, f"{year}")
        if not os.path.exists(out_data_dir):
            os.mkdir(out_data_dir)

        # Run Query to find data for the query_bbox and time period
        roi_query = icepyx.Query('ATL03', query_bbox, [f'{year}-01-01', f'{year}-12-31'])

        print("\t\t", roi_query.avail_granules())

        # Order the ICESAT-2 Granules:
        roi_query.order_granules()
        # Download the ICESAT-2 Granules to out_data_dir 
        roi_query.download_granules(out_data_dir)
        